# ChatBot Project
This Chatbot project is intended for Human AI Class of University of Nottingham. Expect a fully running code with some markdowns to make it clear.

*Disclaimer: This project uses Virtual Environment. Please download the requirements before running the program*

## Program Name: Costa Coffee ChatBot

The program is intended to take orders from customer, with a Natural Language (in English) as the input of the order.

# JSON File

`intents.json` is a JSON file containing the list of intents both for general talk and to take orders. This will serve as a decision tree for further processesing. 

In [1]:
# Get the data from JSON
import json

# Read the json file
intentFile = open('./data/intent.json').read()
intents = json.loads(intentFile)

# Functions

`text_to_vector` function to split a sentence to the base words. After the splitting process, it will return a list of words, alongside with the cont next to it

`get_cosine` function is to calculate the cosine similarity between two vectors. *This function will be used after using `text_to_vector` function.

In [2]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")

# Calculate the cosine similarity between two vectors
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

# Return the text to vector
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [3]:
# Getting all the 

intentDict = {}
for i in range(len(intents['intents'])):
    for j in intents['intents'][i]['patterns']:
        intentDict[intents['intents'][i]['function']] = text_to_vector(' '.join(intents['intents'][i]['patterns']))

print(intentDict)

{'hello': Counter({'hi': 2, 'up': 2, 'hello': 1, 'there': 1, 'good': 1, 'morning': 1, 'what': 1, 's': 1, 'whats': 1}), 'weather': Counter({'hows': 1, 'the': 1, 'weather': 1, 'today': 1}), 'coffee': Counter({'coffee': 1})}


In [4]:
# Check which intent 
text = 'hello there whats the weather today?'
vectorTest = text_to_vector(text)
for l in intentDict.keys():
    print(get_cosine(vectorTest, intentDict[l]))

0.31622776601683794
0.6123724356957946
0.0


In [ ]:
# Defining replies

def hello():
    return "hello"

def weather():
    return "weather function"

def coffee():
    return "coffee heheh"

In [ ]:
# Get the highest similarity, then return tag
from operator import itemgetter

text = 'i want some coffee please'

# Get the highest intent, return the key from dictionary
vectorTest = text_to_vector(text)
closestVector = sorted([[get_cosine(vectorTest, intentDict[l]),l] for l in intentDict.keys()], key=itemgetter(1))

# # With the key, get the tag from json

eval(closestVector[0][1]+"()")